In [ ]:
%matplotlib inline
import os
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt
c=3.e8 #speed of light
import time
filepath='D:/measuring/'
import scipy.signal
os.chdir(filepath)

sys.path.append('H:/My Documents/measuring/')

data_dir='D:\measuring\data/20160719'#:\measuring\data/20160503\ON diamond\pos 7/raw data spectrometer\L9'#'D:\measuring\data/20160707\CL2_newmirror_processed'#"/Users/suzannevandam/Documents/PhD/localdata/20160430/ON_diamond" 
print data_dir

%matplotlib inline

In [ ]:
import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as sa_2D

In [ ]:
nr_datafiles = 9
minimum_peak_heights = [650,8000,1500,800,1000,10000,5000,3500,1000]
avg_nu_fsrs = np.zeros((nr_datafiles,2))
min_fsrs = np.ones(nr_datafiles)*4*1.e12
max_fsrs = np.ones(nr_datafiles)*20.e12
max_fsrs[5] = 40.e12
max_fsrs[6] = 40.e12
max_fsrs[7] = 50.e12
max_fsrs[8] = 50.e12

min_fsrs[7] = 10.e12
min_fsrs[8] = 10.e12

reload(sa_2D)

for i in np.arange(nr_datafiles):
    #data_folder = os.path.join(data_dir,'L'+str(i))
    data_folder = os.path.join(data_dir)
    #note that what are called wavelengths, are actually the frequencies in THz! 
    wavelengths,filenumbers,intensities = sa.load_data_from_folder(data_folder)

    minimum_peak_height = minimum_peak_heights[i]
    x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],
                                       data_dir=data_folder,save_fig=False,
                                       plot_fit =False, plot_peak_locations=True,
                                       minimum_peak_height=minimum_peak_height, g_gamma=1)
    # reinitialise every time
    nu_fsrs=np.array([])

    if i == 4:
        x0s=x0s[:-2]
    
    for l in np.arange(len(x0s)-1):
        fund_1 = x0s[l]
        fund_2 = x0s[l+1]
        nu_fsr = np.abs(fund_2 - fund_1)*1.e12
        if ((nu_fsr>min_fsrs[i]) and (nu_fsr<max_fsrs[i])):
            length = c/(2*nu_fsr)
            print nu_fsr*1.e-12, 'THz'
            print length *1.e6 , 'um'
            nu_fsrs = np.append(nu_fsrs,nu_fsr)
            
    print nu_fsrs
    avg_nu_fsr = np.average(nu_fsrs)
    u_avg_nu_fsr = np.std(nu_fsrs)
    
    avg_nu_fsrs[i,0] = avg_nu_fsr 
    avg_nu_fsrs[i,1] = u_avg_nu_fsr
            
        
#     fund_1 = x0s[0]
#     fund_2 = x0s[3]
#     dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
#     if dnu_fsr>2*1.e12:
#         length = c/(2*dnu_fsr)
#         print dnu_fsr*1.e-12, 'THz'
#         print length *1.e6 , 'um'



# wavelengths,filenumbers,intensities = sa.load_data(data_file)


f = open(os.path.join(data_dir,'fsr_data.txt'), 'w')
np.savetxt(f,avg_nu_fsrs)
f.close()

In [ ]:

nu_fsrs = np.loadtxt(os.path.join(data_dir,'fsr_data.txt'))

linewidths = np.loadtxt(os.path.join(data_dir,'linewidth_data.txt'))



In [ ]:
fig,ax = plt.subplots()

ax.errorbar(nu_fsrs[:,0]*1.e-14,linewidths[:,0],xerr = nu_fsrs[:,1]*1.e-14,yerr=linewidths[:,1])
ax.set_xlabel('linewidth (GHz)')
ax.set_ylabel('nu FSR (THz)')

plt.show()

fig1,ax1 = plt.subplots()

ax1.errorbar(c/(2*nu_fsrs[:,0])*1.e6,np.divide(nu_fsrs[:,0],linewidths[:,0]*1.e9),yerr=linewidths[:,1])
ax1.set_xlabel('cavity length (um)')
ax1.set_ylabel('Finesse')


In [ ]:
reload(sa_2D)
minimum_peak_height = 5000
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],data_dir=data_dir,save_fig=False,plot_fit =False, plot_peak_locations=True,minimum_peak_height=minimum_peak_height, g_gamma=1)




In [ ]:
#Finding cavity length from peaks

for i in np.arange(len(x0s)-1):
    fund_1 = x0s[i]
    fund_2 = x0s[i+1]
    dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
    if dnu_fsr>3*1.e12:
        length = c/(2*dnu_fsr)
        print dnu_fsr*1.e-12, 'THz'
        print length *1.e6 , 'um'


fund_1 = x0s[0]
fund_2 = x0s[3]
dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
if dnu_fsr>2*1.e12:
    length = c/(2*dnu_fsr)
    print dnu_fsr*1.e-12, 'THz'
    print length *1.e6 , 'um'


In [ ]:
reload(sa_2D)
sa_2D.peaks_from_2D_data(data_dir,plot_peak_locations=False,minimum_peak_height=minimum_peak_height)